In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

%matplotlib inline

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_region_of_interest(img, vertices):
    
    #vertices = np.array([[left_bottom, left_top, right_top, right_bottom]], dtype=np.int32)

    color = [0,255,0]
    
    left_bottom = (vertices[0][0][0], vertices[0][0][1])
    left_top = (vertices[0][1][0], vertices[0][1][1])
    right_top = (vertices[0][2][0], vertices[0][2][1])
    right_bottom = (vertices[0][3][0], vertices[0][3][1])

    bottom_center = ((right_bottom[0]+left_bottom[0])//2, right_bottom[1])
    top_center = ((right_top[0]+left_top[0])//2, right_top[1])

    
    cv2.line(img, left_bottom, left_top, color, 2)
    cv2.line(img, left_top, right_top, color, 2)
    cv2.line(img, right_top, right_bottom, color, 2)
    cv2.line(img, right_bottom, left_bottom, color, 2)
    cv2.line(img, bottom_center, top_center, color, 2)
    

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns array of hough lines.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines
    
def lines_img(img, lines):    
    """
    `img` is an image with desired shape
        
    Returns a blank image with lines drawn on it.
    """
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the lines_img(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape! TODO
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def line_coeffs(line):
    """
    from known end points (x0,y0), (x1,y1) of a line segment solve c0 and c1 for equation y = c0 + c1*x.
    line == [x0, y0, x1, y1]
    """
    c1 = (line[3]-line[1])/(line[2]-line[0]) # slope
    c0 = line[1] - c1 * line[0]
    return c0, c1

def line_length(line):
    a = np.square([line[2]-line[0]])[0]
    b = np.square([line[2]-line[0]])[0]
    
    return np.sqrt([a+b])[0]
    
def extrapolate_left_right(lines, center, bottom, top):

    right_lines = []
    right_lengths = []
    left_lines = []
    left_lengths = []

    for line in lines:
        c0, c1 = line_coeffs(line[0])
        len = line_length(line[0])
        if c1 > 0 and line[0][2] > center:
            right_lines.append((c0,c1))
            right_lengths.append(len)
        elif c1 < 0 and line[0][2] < center:
            left_lines.append((c0,c1))
            left_lengths.append(len)
            
    ravg = np.average(right_lines, axis=0, weights=right_lengths)
    lavg = np.average(left_lines, axis=0, weights=left_lengths)
    
    # now calculate end points for left and right line segments
    left_line = np.array([[(bottom-lavg[0])/lavg[1],bottom,(top-lavg[0])/lavg[1],top]], dtype=np.int32)
    right_line = np.array([[(bottom-ravg[0])/ravg[1],bottom,(top-ravg[0])/ravg[1],top]], dtype=np.int32)

    return left_line, right_line

In [3]:
class Parameters:
    def __init_(self):
        # Gaussian blur kernel size
        self.kernel_size = None
        # Canny transform thresholds
        self.low_threshold = None
        self.high_threshold = None
        # Area of interest
        self.left_bottom = None
        self.left_top = None
        self.right_bottom = None
        self.right_top = None
        # Hough transform parameters
        self.rho = None           # distance resolution in pixels of the Hough grid
        self.theta = None         # angular resolution in radians of the Hough grid
        self.threshold = None     # minimum number of votes (intersections in Hough grid cell)
        self.min_line_len = None  # minimum number of pixels making up a line
        self.max_line_gap = None  # maximum gap in pixels between connectable line segments
        # Run extrapolation for line segments
        self.extrapolate = None
        

def find_lane_lines(img, p):
                    
    gray = grayscale(img)
    
    # Bluring and Canny transform
    blur_gray = gaussian_blur(gray, p.kernel_size)
    edges = canny(blur_gray, p.low_threshold, p.high_threshold)
  
    # region of interest
    vertices = np.array([[p.left_bottom, p.left_top, p.right_top, p.right_bottom]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # Hough lines
    lines = hough_lines(masked_edges, p.rho, p.theta, p.threshold, p.min_line_len, p.max_line_gap)
    if p.extrapolate:
        left_line, right_line = extrapolate_left_right(lines, (p.right_top[0]+p.left_top[0])/2, imshape[0], p.left_top[1])
        line_image = lines_img(masked_edges, [left_line, right_line])
    else:
        line_image = lines_img(masked_edges, lines) # show 'raw' hough lines  
    
    combined_image = weighted_img(line_image, img)
    
    if p.show_region_of_interest:
        draw_region_of_interest(combined_image, vertices)
    
    #return combined_image
    return edges

def process_image(img):
    
    p = Parameters()
    
    p.kernel_size = 7
    p.low_threshold = 50
    p.high_threshold = 150

    imshape = img.shape
    p.left_bottom = (80, imshape[0]-20)
    p.left_top = (420, 330)
    p.right_bottom = (imshape[1], imshape[0]-20)
    p.right_top = (560, 330)
    
    p.rho = 2
    p.theta = np.pi/180
    p.threshold = 15
    p.min_line_len = 5
    p.max_line_gap = 1
    p.extrapolate = True
    p.show_region_of_interest = False
    
    return find_lane_lines(img, p)

def process_challenge(img):
    
    p = Parameters()
    
    p.kernel_size = 7
    p.low_threshold = 50
    p.high_threshold = 150
    
    img_shape = img.shape # 720 x 1280
    p.left_bottom = (100, img_shape[0]-55)
    p.left_top = (img_shape[1]//2-50, 440)
    p.right_bottom = ( img_shape[1]-70,  img_shape[0]-55)
    p.right_top = (img_shape[1]//2+100, 440)
    
    p.rho = 2
    p.theta = np.pi/180
    p.threshold = 15
    p.min_line_len = 5
    p.max_line_gap = 1
    p.extrapolate = False
    p.show_region_of_interest = True
    
    return find_lane_lines(img, p)

    

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:06<00:00, 41.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 3.98 s, sys: 902 ms, total: 4.88 s
Wall time: 7.83 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))